In [1]:
import numpy as np
import pandas as pd
import re
from contractions import fix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_label = pd.read_csv('test_labels.csv')

In [3]:
train[:20]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
def clean_text(text):
    """
    Clean input text by removing URLs, HTML tags, non-ASCII characters, special characters,
    emojis, punctuation, and line breaks.
    """
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

    # Remove HTML tags and entities
    text = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', ' ', text)

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7f]', ' ', text)

    # Remove special characters and emojis
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    text = emoji_pattern.sub(' ', text)

    # Remove punctuation
    text = re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', ' ', text)

    # Remove line breaks
    text = re.sub(r'\n+|\r+', ' ', text)

    #fix contractions such as 'can't' to 'cannot'
    text = fix(text)

    return text


In [5]:
train, valid = train_test_split(train, train_size=0.8, random_state=42)
print(train.shape)
print(valid.shape)

(127656, 8)
(31915, 8)


In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize,
                             min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                             smooth_idf=1, sublinear_tf=1, preprocessor=clean_text)

In [7]:
x = vectorizer.fit_transform(train['comment_text'])

In [8]:
valid_x = vectorizer.transform(valid['comment_text'])
test_x = vectorizer.transform(test['comment_text'])

In [9]:
print(x.shape)
print(valid_x.shape)
print(test_x.shape)

(127656, 354430)
(31915, 354430)
(153164, 354430)


In [10]:
train_labels = train.drop(['id', 'comment_text'], axis=1)
valid_labels = valid.drop(['id', 'comment_text'], axis=1)

classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize storage for models and scores
models = []
scores = []

In [11]:
for col in classes:
    print(f"Processing {col}...")

    # Compute the probability of features given the class
    p = (x[train_labels[col] == 1].sum(0) + 1) / (train_labels[col].sum() + 1)
    q = (x[train_labels[col] == 0].sum(0) + 1) / ((train_labels[col] == 0).sum() + 1)
    r = np.log(p / q)
    #print(r)
    # Train the logistic regression model using feature matrix modified by log-count ratios
    model = LogisticRegression(C=4, solver='liblinear')
    x_nb = x.multiply(r)
    model.fit(x_nb, train_labels[col])

    # Store the model
    models.append((model, r))

    # Validation
    #print(valid_x)
    val_x_nb = valid_x.multiply(r)
    preds = model.predict_proba(val_x_nb)[:, 1].reshape(-1,1)
    auc_score = roc_auc_score(valid_labels[col], preds)
    scores.append(auc_score)

    print(f"ROC AUC for {col}: {auc_score}")

Processing toxic...
ROC AUC for toxic: 0.9777932240597857
Processing severe_toxic...
ROC AUC for severe_toxic: 0.9805322572979568
Processing obscene...
ROC AUC for obscene: 0.986666982024598
Processing threat...
ROC AUC for threat: 0.9808147238347296
Processing insult...
ROC AUC for insult: 0.9807785941360673
Processing identity_hate...
ROC AUC for identity_hate: 0.9718756070784786


In [13]:
# Initialize an array to hold predictions for each class
test_preds = np.zeros((len(test), len(classes)))

# Iterate over each trained model
for i, (model, r) in enumerate(models):
    # Adjust the test features by the learned ratios 'r'
    test_x_nb = test_x.multiply(r)
    # Predict probabilities for the positive class
    test_preds[:, i] = model.predict_proba(test_x_nb)[:, 1]

# Create a DataFrame to hold the results
results_df = pd.DataFrame(test_preds, columns=classes)


In [14]:
results_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.999987,0.099889,0.999984,0.003255,0.995947,0.100477
1,0.007738,0.000844,0.003220,0.000582,0.002698,0.001796
2,0.014164,0.000455,0.004101,0.000147,0.004270,0.000467
3,0.002256,0.000285,0.001699,0.000234,0.000987,0.000277
4,0.008454,0.000475,0.002083,0.000153,0.002166,0.000368
